# 1、data preprocessing

In [ ]:
import pandas as pd
Well_B = pd.read_csv(r'E:\jupyter\lost_circulation\records\paper-bhyt\MLR\data\Well_B.csv')
Well_B.info()
# Original data information

In [ ]:
# Remove missing values
# Remove rows where torque is less than 0 (it was later discovered that there were negative torque values)
Well_B = Well_B[Well_B['扭矩'] >= 0]
Well_B = Well_B.dropna()
Well_B.reset_index(inplace=True, drop=True)  # Reset index and remove the old index column
Well_B.info()
Well_B.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_1.csv', index=False, encoding='utf_8_sig')

In [ ]:
# Perform label encoding for object type variables
# Well number, Well type, Layer, Lithology, Formation structure, Lost circulation status
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def label_encode_columns(input_csv, output_csv, columns_to_encode, mappings_dir):
    # Read the CSV file
    df = pd.read_csv(input_csv)
    
    # Create a dictionary to store the LabelEncoder objects for each column
    label_encoders = {}
    
    # Ensure the directory for saving mapping files exists
    if not os.path.exists(mappings_dir):
        os.makedirs(mappings_dir)

    # Perform label encoding on specified columns
    for column in columns_to_encode:
        le = LabelEncoder()
        df[column + '_encoded'] = le.fit_transform(df[column])
        label_encoders[column] = le
        
        # Save the unique mappings between the original data and the encoded values
        unique_mappings = pd.DataFrame({
            column: le.classes_,
            column + '_encoded': range(len(le.classes_))
        })
        mapping_file = os.path.join(mappings_dir, f'{column}_mappings.csv')
        unique_mappings.to_csv(mapping_file, index=False, encoding='utf_8_sig')

    # Save the encoded data into a new CSV file
    df.to_csv(output_csv, index=False, encoding='utf_8_sig')
    
    return df, label_encoders

# Example usage
input_csv = 'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_1.csv'  # Input CSV file path
output_csv = 'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/output_encoded_label.csv'  # Output CSV file path
mappings_dir = 'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/mappings'  # Directory to save encoding mappings
columns_to_encode = ['井号', '井别','层位','岩性','地层构造','漏失情况']  # List of columns to encode

df_encoded, encoders = label_encode_columns(input_csv, output_csv, columns_to_encode, mappings_dir)
print(df_encoded.head())  # Display the first few rows of the encoded data


In [ ]:
# Select numerical data for saving (convert feature names to English)
df = pd.read_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/output_encoded_label.csv')
df_1 = df[['井号_encoded', '井别_encoded', '井深', '垂深', '层位_encoded', '岩性_encoded',
           '地层构造_encoded', '钻压', '转速', '扭矩', '泵压', '排量', '出口密度', 'ECD', '大钩负荷', '机械钻速',
           '钻时', 'DC指数', '迟到时间', '出口流量', '地层压力梯度', '地层破裂压力梯度', '理论最大排量', '排泵比', '漏失情况_encoded']]
df_1.columns = ['WellName', 'WellType', 'WellDepth', 'TVD', 'Layer', 'Lithology', 'FormationStructure', 'WOB', 'RPM', 'TOR', 'PumpPressure',
                'Displacement', 'Density', 'ECD', 'HookLoad', 'ROP', 'DrillTime', 'DC', 'LagTime', 'OutletFlow', 'FormationPressureGradient',
                'FormationRupturePressureGradient', 'TheoreticalMaximumDisplacement', 'DisPumpRatio', 'LostCirculation']
df_1.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_encoded_label.csv', encoding='utf_8_sig', index=False)
df_1.info()  # Save the encoded data and output the data information
df_2 = df_1.describe()
print(df_2)
df_2.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_encoded_label_describe.csv', encoding='utf_8_sig')
# Save and output the data description information

# anomaly value processing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

# Create a gradient from blue to red
cmap = mcolors.LinearSegmentedColormap.from_list("blue_red", ["blue", "red"])

# Set plotting parameters 
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 12  # Set the overall font size to 12, suitable for most papers
plt.rcParams['font.weight'] = 'bold'  # Set overall font to bold
# Set individual settings for axis labels, titles, legends, and ticks
plt.rcParams['axes.titlesize'] = 14  # Larger axis title
plt.rcParams['axes.labelsize'] = 12   # Slightly larger axis labels
plt.rcParams['axes.titleweight'] = 'bold'  # Bold axis titles
plt.rcParams['axes.labelweight'] = 'bold'  # Bold axis labels
plt.rcParams['xtick.labelsize'] = 10   # Slightly larger x-axis tick labels
plt.rcParams['ytick.labelsize'] = 10   # Slightly larger y-axis tick labels
plt.rcParams['legend.fontsize'] = 10   # Larger legend font
plt.rcParams['legend.title_fontsize'] = 12  # Larger legend title font

# Read data
data = pd.read_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_encoded_label.csv')

# Select numeric data
feature_columns = ['WellDepth', 'DC', 'FormationPressureGradient', 'FormationRupturePressureGradient',
                   'WOB', 'RPM', 'TOR', 'PumpPressure', 'HookLoad', 'ROP',
                   'Displacement', 'Density', 'ECD', 'OutletFlow', 'LagTime', 'TheoreticalMaximumDisplacement']
data_des = data[feature_columns].describe()
data_des.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_raw_des.csv', encoding='utf-8-sig')

# Initialize cleaned_data as the original data
cleaned_data = data.copy()

# Loop through each feature column
for feature in feature_columns:
    # Calculate IQR
    Q1 = cleaned_data[feature].quantile(0.25)
    Q3 = cleaned_data[feature].quantile(0.75)
    IQR = Q3 - Q1

    # Define lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Mark anomalies
    cleaned_data['anomaly'] = ((cleaned_data[feature] < lower_bound) | (cleaned_data[feature] > upper_bound)).astype(int)

    # Visualize anomaly detection results (scatter plot)
    plt.figure(figsize=(12, 6))

    # Plot normal data (blue)
    normal_data = cleaned_data[cleaned_data['anomaly'] == 0]
    plt.scatter(normal_data.index, normal_data[feature], color='blue', label='Normal Data', alpha=0.6)

    # Plot anomaly data (red)
    anomaly_data = cleaned_data[cleaned_data['anomaly'] == 1]
    plt.scatter(anomaly_data.index, anomaly_data[feature], color='red', label='Anomaly Data', alpha=0.6)

    # Apply gradient color map to the entire dataset
    scatter = plt.scatter(cleaned_data.index, cleaned_data[feature], c=cleaned_data['anomaly'], cmap=cmap, alpha=0.6, vmin=0, vmax=1)

    plt.title(f'IQR Anomaly Detection for {feature}')
    plt.xlabel('Index')
    plt.ylabel(feature)
    plt.axhline(0, color='grey', lw=1)
    plt.grid(True)

    # Add color bar and set labels
    cbar = plt.colorbar(scatter)  # Create color bar
    cbar.set_label('Anomaly Label (0: Normal, 1: Anomaly)')  # Set color bar label
    cbar.set_ticks([0, 1])  # Set ticks for color bar
    cbar.set_ticklabels(['Normal', 'Anomaly'])  # Set tick labels for color bar
    # Adjust subplot margins to avoid extra space on the right
    plt.subplots_adjust(right=1)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/IQR/detection_{feature}.png', dpi=300)
    plt.show()

    # Remove outliers
    cleaned_data = cleaned_data[cleaned_data['anomaly'] == 0]
    # Merge original data and cleaned data into one DataFrame
    combined_data = pd.DataFrame({
        'Original': data[feature],
        'Cleaned': cleaned_data[feature].reindex(data.index)  # Maintain consistent index
    })
    # Visualize comparison of original and cleaned data distributions (box plot)
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=combined_data, palette='pastel')
    plt.title(f'Comparison of Original and Cleaned Data for {feature}')
    plt.ylabel(feature)
    plt.xticks(ticks=[0, 1], labels=['Original', 'Cleaned'])
    plt.tight_layout()
    plt.savefig(f'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/IQR/comparison_{feature}.png', dpi=300)
    plt.show()

    # Clean temporary column
    cleaned_data.drop(columns=['anomaly'], inplace=True)

# Save the cleaned data after removing outliers
cleaned_data.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_IQR.csv', encoding='utf-8-sig', index=False)


In [ ]:
# Get descriptive statistics of the data after handling outliers
IQR = pd.read_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_IQR.csv')
IQR_des = IQR.describe()
print(IQR_des)
IQR_des.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_IQR_des.csv', encoding='utf-8-sig')


# 2、feature importance

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

# Set plotting parameters
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False
# Set font to bold to meet SCI paper requirements
plt.rcParams['font.size'] = 12  # Set overall font size to 12, suitable for most papers
plt.rcParams['font.weight'] = 'bold'  # Set overall font to bold
# Set individual settings for axis labels, titles, legends, and ticks
plt.rcParams['axes.titlesize'] = 14  # Larger axis titles
plt.rcParams['axes.labelsize'] = 12   # Slightly larger axis labels
plt.rcParams['axes.titleweight'] = 'bold'  # Bold axis titles
plt.rcParams['axes.labelweight'] = 'bold'  # Bold axis labels
plt.rcParams['xtick.labelsize'] = 10   # Slightly larger x-axis tick labels
plt.rcParams['ytick.labelsize'] = 10   # Slightly larger y-axis tick labels
plt.rcParams['legend.fontsize'] = 10   # Larger legend font
plt.rcParams['legend.title_fontsize'] = 12  # Larger legend title font

# 1. Data preparation
# Import data from CSV file
data = pd.read_csv(r'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_IQR.csv')

# Preview the first few rows to confirm data structure
print("Data preview:")
print(data.head())

# Select feature columns and target label column
# Assuming the target column is 'LostCirculation'
feature_columns = ['WellDepth', 'DC', 'FormationPressureGradient', 'FormationRupturePressureGradient',
                   'WOB', 'RPM', 'TOR', 'PumpPressure', 'HookLoad', 'ROP',
                   'Displacement', 'Density', 'ECD', 'OutletFlow', 'LagTime', 'TheoreticalMaximumDisplacement']
target_column = 'LostCirculation'

# Extract features and labels
X = data[feature_columns]
y = data[target_column]
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Normalize data
scaler = MinMaxScaler()
# Normalize only the training features and apply it to both training and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print("\nRandom Forest Model Performance:")
print(f"Accuracy: ", rf_accuracy)
print(classification_report(y_test, y_pred_rf))

# 3. XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_scaled, y_train)
y_pred_xgb = xgb_model.predict(X_test_scaled)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print("\nXGBoost Model Performance:")
print(f"Accuracy: ", xgb_accuracy)
print(classification_report(y_test, y_pred_xgb))

# 4. LightGBM model
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train_scaled, y_train)
y_pred_lgb = lgb_model.predict(X_test_scaled)
lgb_accuracy = accuracy_score(y_test, y_pred_lgb)
print("\nLightGBM Model Performance:")
print(f"Accuracy: ", lgb_accuracy)
print(classification_report(y_test, y_pred_lgb))

# 5. Extract feature importance from each model
# Random Forest
rf_importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
# XGBoost
xgb_importance = pd.Series(xgb_model.feature_importances_, index=X.columns).sort_values(ascending=False)
# LightGBM
lgb_importance = pd.Series(lgb_model.feature_importances_, index=X.columns).sort_values(ascending=False)

# 6. Save feature importance to a CSV file
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'RandomForest': rf_importance.reindex(X.columns).values,
    'XGBoost': xgb_importance.reindex(X.columns).values,
    'LightGBM': lgb_importance.reindex(X.columns).values,
    'RF_accuracy': accuracy_score(y_test, y_pred_rf),
    'XGB_accuracy': accuracy_score(y_test, y_pred_xgb),
    'LGBM_accuracy': accuracy_score(y_test, y_pred_lgb)
})
# Save to CSV file
importance_df.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_feature_importance.csv', index=False, encoding='utf-8-sig')
print("Feature importance saved to 'Well_B_feature_importance.csv'")

# 7. Plot feature importance bar charts for each model and save them
# 7.1 Random Forest Feature Importance
plt.figure(figsize=(8, 6))
sns.barplot(x=rf_importance.values, y=rf_importance.index, color='skyblue')
for index, value in enumerate(rf_importance.values):
    plt.text(value, index, f'{value:.2f}', va='center')  # Annotate values on the bar chart
plt.title("Random Forest Feature Importance")
plt.xlabel("Importance score")
plt.ylabel("Feature")
# Save image
plt.tight_layout()
plt.savefig('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/RF_feature_importance.png', dpi=300)
print("Random Forest feature importance chart saved as 'RF_feature_importance.png'")
plt.show()

# 7.2 XGBoost Feature Importance
plt.figure(figsize=(8, 6))
sns.barplot(x=xgb_importance.values, y=xgb_importance.index, color='lightgreen')
for index, value in enumerate(xgb_importance.values):
    plt.text(value, index, f'{value:.2f}', va='center')
plt.title("XGBoost Feature Importance")
plt.xlabel("Importance score")
plt.ylabel("Feature")
# Save image
plt.tight_layout()
plt.savefig('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/XGB_feature_importance.png', dpi=300)
print("XGBoost feature importance chart saved as 'XGB_feature_importance.png'")
plt.show()

# 7.3 LightGBM Feature Importance
plt.figure(figsize=(8, 6))
sns.barplot(x=lgb_importance.values, y=lgb_importance.index, color='lightcoral')
for index, value in enumerate(lgb_importance.values):
    plt.text(value, index, f'{value:.2f}', va='center')
plt.title("LightGBM Feature Importance")
plt.xlabel("Importance score")
plt.ylabel("Feature")
# Save image
plt.tight_layout()
plt.savefig('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/LGBM_feature_importance.png', dpi=300)
print("LightGBM feature importance chart saved as 'LGBM_feature_importance.png'")
plt.show()

# 8. Combine feature importance across models
# 8.1 Normalize feature importance to 0-1 range
rf_importance_norm = (rf_importance - rf_importance.min()) / (rf_importance.max() - rf_importance.min())
xgb_importance_norm = (xgb_importance - xgb_importance.min()) / (xgb_importance.max() - xgb_importance.min())
lgb_importance_norm = (lgb_importance - lgb_importance.min()) / (lgb_importance.max() - lgb_importance.min())

# 8.2 Calculate combined feature importance (using arithmetic or geometric mean)
# Convert feature importance to ranks (1 being the highest)
# 1. Ensure index alignment to avoid feature loss
all_features = rf_importance.index.union(xgb_importance.index).union(lgb_importance.index)
rf_importance = rf_importance.reindex(all_features, fill_value=0)
xgb_importance = xgb_importance.reindex(all_features, fill_value=0)
lgb_importance = lgb_importance.reindex(all_features, fill_value=0)

# 2. Calculate feature rank for each model (smaller value means higher rank)
rf_rank = rf_importance.rank(ascending=False)
xgb_rank = xgb_importance.rank(ascending=False)
lgb_rank = lgb_importance.rank(ascending=False)

# 3. Calculate reverse rank weights: 1 - (Rank - 1) / n
n = len(all_features)
reverse_rf_rank = 1 - (rf_rank - 1) / n
reverse_xgb_rank = 1 - (xgb_rank - 1) / n
reverse_lgb_rank = 1 - (lgb_rank - 1) / n

# 4. Multiply reverse rank weights by model accuracies
weighted_rf = reverse_rf_rank * rf_accuracy
weighted_xgb = reverse_xgb_rank * xgb_accuracy
weighted_lgb = reverse_lgb_rank * lgb_accuracy

# 5. Calculate final combined importance (average)
combined_importance = (weighted_rf + weighted_xgb + weighted_lgb) / 3

# 6. Store results in a DataFrame
final_importance_df = pd.DataFrame({
    'Feature': all_features,
    'RF_Rank': rf_rank.values,
    'XGB_Rank': xgb_rank.values,
    'LGB_Rank': lgb_rank.values,
    'Reverse_RF_Rank': reverse_rf_rank.values,
    'Reverse_XGB_Rank': reverse_xgb_rank.values,
    'Reverse_LGB_Rank': reverse_lgb_rank.values,
    'CombinedImportance': combined_importance.values
})

# 7. Output final results
print(final_importance_df)

# 8.3 Save combined feature importance to CSV file
final_importance_df.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_combined_feature_importance.csv', index=False, encoding='utf-8-sig')
print("Combined feature importance saved to 'Well_B_combined_feature_importance.csv'")

# 8.4 Plot combined feature importance bar chart and save (sorted in descending order)
final_importance_df_sorted = final_importance_df.sort_values(by='CombinedImportance', ascending=False)
plt.figure(figsize=(10, 8))
# Use sns.barplot to plot sorted bar chart
sns.barplot(x='CombinedImportance', y='Feature', data=final_importance_df_sorted, palette='coolwarm')
# Annotate each bar with its value
for index, value in enumerate(final_importance_df_sorted['CombinedImportance']):
    plt.text(value, index, f'{value:.2f}', va='center')
# Set chart title and axis labels
plt.title("Combined Feature Importance")
plt.xlabel("Normalized importance score")
plt.ylabel("Feature")
# Save image
plt.tight_layout()
plt.savefig('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/Combined_feature_importance.png', dpi=300)
print("Combined feature importance chart (sorted) saved as 'Combined_feature_importance.png'")
# Show chart
plt.show()


# 3、MLR model

In [ ]:
import pandas as pd  
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Set plot parameters
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False
# Set font to bold to meet SCI paper requirements
plt.rcParams['font.size'] = 12  # Set font size to 12, suitable for most papers
plt.rcParams['font.weight'] = 'bold'  # Set font weight to bold
# Separate settings for axis labels, titles, legends, and ticks
plt.rcParams['axes.titlesize'] = 14  # Larger axis titles
plt.rcParams['axes.labelsize'] = 12   # Slightly larger axis labels
plt.rcParams['axes.titleweight'] = 'bold'  # Bold axis titles
plt.rcParams['axes.labelweight'] = 'bold'  # Bold axis labels
plt.rcParams['xtick.labelsize'] = 10   # Slightly larger x-axis tick labels
plt.rcParams['ytick.labelsize'] = 10   # Slightly larger y-axis tick labels
plt.rcParams['legend.fontsize'] = 10   # Larger legend font
plt.rcParams['legend.title_fontsize'] = 12  # Larger legend title font


# Read data
df = pd.read_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_IQR.csv')

# Define feature lists
alpha_keys = ['WellDepth', 'DC', 'FormationPressureGradient', 'FormationRupturePressureGradient']
beta_keys = ['WOB', 'RPM', 'TOR', 'PumpPressure', 'HookLoad', 'ROP']
gamma_keys = ['Displacement', 'Density', 'ECD', 'OutletFlow', 'LagTime', 'TheoreticalMaximumDisplacement']

# Feature normalization function
def normalize(column):
    return (df[column] - df[column].min()) / (df[column].max() - df[column].min())

# Normalize features
df['WD_n'] = normalize('WellDepth')
df['DC_n'] = normalize('DC')
df['FoPG_n'] = normalize('FormationPressureGradient')
df['FrPG_n'] = normalize('FormationRupturePressureGradient')
df['WOB_n'] = normalize('WOB')
df['RPM_n'] = normalize('RPM')
df['TOR_n'] = normalize('TOR')
df['PP_n'] = normalize('PumpPressure')
df['HL_n'] = normalize('HookLoad')
df['ROP_n'] = normalize('ROP')
df['DIS_n'] = normalize('Displacement')
df['OD_n'] = normalize('Density')
df['ECD_n'] = normalize('ECD')
df['FO_n'] = normalize('OutletFlow')
df['LT_n'] = normalize('LagTime')
df['TMD_n'] = normalize('TheoreticalMaximumDisplacement')

# Define the mapping of normalized column names
normalized_columns_mapping = {
    'WellDepth': 'WD_n',
    'DC': 'DC_n',
    'FormationPressureGradient': 'FoPG_n',
    'FormationRupturePressureGradient': 'FrPG_n',
    'WOB': 'WOB_n',
    'RPM': 'RPM_n',
    'TOR': 'TOR_n',
    'PumpPressure': 'PP_n',
    'HookLoad': 'HL_n',
    'ROP': 'ROP_n',
    'Displacement': 'DIS_n',
    'Density': 'OD_n',
    'ECD': 'ECD_n',
    'OutletFlow': 'FO_n',
    'LagTime': 'LT_n',
    'TheoreticalMaximumDisplacement': 'TMD_n',
}

# Data centering function
def center_data(df, keys):
    return df[keys] - df[keys].mean()

# Read weights from CSV file and align with feature names
def load_weights(file_path, feature_keys):
    weights_df = pd.read_csv(file_path)
    weights_df.set_index('Feature', inplace=True)  # Ensure feature names are set as index
    # Extract weights corresponding to feature_keys
    weights = weights_df.loc[feature_keys, 'CombinedImportance'].values
    return weights

# Fit logistic regression model
def fit_logistic_regression(keys, normalized_columns_mapping, weights, file_path, C=5000):
    # Prepare data and center it
    X = center_data(df, [normalized_columns_mapping[col] for col in keys])
    y = df['LostCirculation']
    # Apply feature weights
    weights = load_weights(file_path, keys)
    X_weighted = X * weights
    # Fit model without adding the constant term
    model = sm.Logit(y, X_weighted)
    results = model.fit(disp=0)  # Suppress output during fitting
    # Get coefficients
    params = results.params
    # Get the smallest positive coefficient
    if (params > 0).any():  # If any coefficient is greater than 0
        min_positive_param = params[params > 0].min()
    else:
        # If all coefficients are ≤ 0, use a default value (e.g., the minimum coefficient)
        min_positive_param = params.min()

    # Replace coefficients ≤ 0 with the smallest positive coefficient
    params = np.where(params <= 0, min_positive_param, params)
    # Predict probabilities
    predictions = results.predict(X_weighted)
    
    return predictions, params  # Return predictions and adjusted coefficients

file_path = 'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_combined_feature_importance.csv'
# Calculate risk index and coefficients
df['FormationRisk'], formation_params = fit_logistic_regression(alpha_keys, normalized_columns_mapping, None, file_path)
df['DrillRisk'], drill_params = fit_logistic_regression(beta_keys, normalized_columns_mapping, None, file_path)
df['FluidRisk'], fluid_params = fit_logistic_regression(gamma_keys, normalized_columns_mapping, None, file_path)

# Round risk index to 3 decimal places
df['FormationRisk'] = round(df['FormationRisk'], 3)
df['DrillRisk'] = round(df['DrillRisk'], 3)
df['FluidRisk'] = round(df['FluidRisk'], 3)

# Plot fitted vs actual and annotate formula
def plot_fitted_vs_actual(risk_name, predictions, actual, title, formula):
    plt.figure(figsize=(10, 6))
    # Plot actual values
    plt.scatter(actual, actual, color='blue', label='LostCirculation', alpha=0.6)
    # Plot risk index
    plt.scatter(actual, predictions, color='red', label='RiskIndex', alpha=0.6)
    # Add perfect fit line
    plt.plot([min(actual), max(actual)], [min(actual), max(actual)], color='green', linestyle='--', label='Perfect Fit')
    
    # Add title and labels
    plt.title(title)
    plt.xlabel('LostCirculation')
    plt.ylabel('RiskIndex')
    plt.legend(loc='lower right')
    plt.grid()

    # Display the risk index formula
    plt.text(0.015, 0.9, formula, transform=plt.gca().transAxes,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8), fontsize=10)
    plt.tight_layout()
    plt.savefig(f'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/fit_{risk_name}.png', dpi=300)
    plt.show()

# Define risk calculation formula
def create_formula(params, keys):
    formula = "Risk = "
    for i, key in enumerate(keys):
        if i < len(params):  # Ensure index is within range
            if params[i] >= 0 and i > 0:  # Don't add plus sign for the first coefficient
                formula += f" + {params[i]:.1f} * {key}"
            else:
                formula += f" {params[i]:.1f} * {key}"
    return formula

formation_formula = create_formula(formation_params, alpha_keys)
drill_formula = create_formula(drill_params, beta_keys)
fluid_formula = create_formula(fluid_params, gamma_keys)

# Plot fitted vs actual
plot_fitted_vs_actual('FormationRisk', df['FormationRisk'], df['LostCirculation'], 'FormationRisk - LostCirculation', formation_formula)
plot_fitted_vs_actual('DrillRisk', df['DrillRisk'], df['LostCirculation'], 'DrillRisk - LostCirculation', drill_formula)
plot_fitted_vs_actual('FluidRisk', df['FluidRisk'], df['LostCirculation'], 'FluidRisk - LostCirculation', fluid_formula)

# Function to compute confusion matrix and ROC curve
def evaluate_risk(risk_values, actual, risk_name):
    threshold = 0.55  # Set threshold
    df[f'Predicted_{risk_name}'] = (risk_values >= threshold).astype(int)

    # Compute confusion matrix
    cm = confusion_matrix(actual, df[f'Predicted_{risk_name}'])
    print(f"Confusion Matrix for {risk_name}:")
    print(cm)

    # Compute AUC
    auc = roc_auc_score(actual, risk_values)
    print(f"AUC for {risk_name}: {auc:.3f}")

    # Plot ROC curve
    fpr, tpr, thresholds = roc_curve(actual, risk_values)
    plt.plot(fpr, tpr, label=f'{risk_name} ROC curve (AUC = {auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random classifier')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {risk_name}')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

# Evaluate the risk models
evaluate_risk(df['FormationRisk'], df['LostCirculation'], 'FormationRisk')
evaluate_risk(df['DrillRisk'], df['LostCirculation'], 'DrillRisk')
evaluate_risk(df['FluidRisk'], df['LostCirculation'], 'FluidRisk')

# Calculate the comprehensive loss circulation risk index
# Read feature weight data
weights_df = pd.read_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_combined_feature_importance.csv')
weights_df.set_index('Feature', inplace=True)

# Extract weights for each category
alpha_weights = weights_df.loc[alpha_keys, 'CombinedImportance']
beta_weights = weights_df.loc[beta_keys, 'CombinedImportance']
gamma_weights = weights_df.loc[gamma_keys, 'CombinedImportance']

# Calculate the proportion of each feature weight in the overall feature weight for each category
total_weight = alpha_weights.sum() + beta_weights.sum() + gamma_weights.sum()
formation_weight = alpha_weights.sum() / total_weight
drill_weight = beta_weights.sum() / total_weight
fluid_weight = gamma_weights.sum() / total_weight

# Normalize the weights for each category
alpha_weights = alpha_weights / alpha_weights.sum()
beta_weights = beta_weights / beta_weights.sum()
gamma_weights = gamma_weights / gamma_weights.sum()

# Use dynamic weights to calculate the final loss circulation risk index (MLR)
# The weights here can be set according to specific needs
df['MLR'] = round(formation_weight * df['FormationRisk'] + drill_weight * df['DrillRisk'] + fluid_weight * df['FluidRisk'], 3)

# Save the new CSV file
df.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_MLR.csv', index=False, encoding='utf-8-sig')

# View the relationship between MLR and loss circulation
print(df[['MLR', 'LostCirculation']].describe())

# Define the MLR column
original_mlr_zero = df[df['LostCirculation'] == 0]['MLR']
original_mlr_one = df[df['LostCirculation'] == 1]['MLR']

# Visualize the distribution comparison of the original data and loss circulation risk index
def plot_original_mlr_distribution(original_zero, original_one, threshold, save_path):
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))  # Use a 1x2 subplot layout

    # Distribution plot for the original data (normal circulation category)
    sns.histplot(original_zero, bins=30, kde=True, color='#1f77b4', ax=axes[0])
    axes[0].set_title('Normal Circulation (0)')
    axes[0].set_xlabel('Machine-learning Lost-circulation Risk-index (MLR)')
    axes[0].set_ylabel('Frequency')

    # Mark the part below the threshold
    axes[0].axvline(x=threshold, color='orange', linestyle='--', label='Threshold')
    axes[0].fill_betweenx([0, axes[0].get_ylim()[1]], axes[0].get_xlim()[0], threshold, color='yellow', alpha=0.3)
    axes[0].text(threshold, axes[0].get_ylim()[1] * 0.95, 'Below Threshold', color='black')

    for p in axes[0].patches:
        height = p.get_height()
        if height > 0:
            axes[0].text(p.get_x() + p.get_width() / 2., height, f'{int(height)}', ha='center', va='bottom')

    # Distribution plot for the original data (loss circulation category)
    sns.histplot(original_one, bins=30, kde=True, color='red', ax=axes[1])
    axes[1].set_title('Lost Circulation (1)')
    axes[1].set_xlabel('Machine-learning Lost-circulation Risk-index (MLR)')
    axes[1].set_ylabel('Frequency')

    # Mark the part above the threshold
    axes[1].axvline(x=threshold, color='orange', linestyle='--', label='Threshold')
    axes[1].fill_betweenx([0, axes[1].get_ylim()[1]], threshold, axes[1].get_xlim()[1], color='yellow', alpha=0.3)
    axes[1].text(threshold, axes[1].get_ylim()[1] * 0.95, 'Above Threshold', color='black')

    for p in axes[1].patches:
        height = p.get_height()
        if height > 0:
            axes[1].text(p.get_x() + p.get_width() / 2., height, f'{int(height)}', ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.show()

# Call the function to draw the MLR-LC comparison
threshold_value = 0.55  # Set threshold
save_path = 'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/comparison_mlr.png'
plot_original_mlr_distribution(original_mlr_zero, original_mlr_one, threshold_value, save_path)

def fit_logistic_model_and_plot(df, mrl_column, target_column, threshold=0.55):
    # Add constant term
    X = sm.add_constant(df[mrl_column])  
    y = df[target_column]
    
    # Fit logistic regression model
    model = sm.Logit(y, X)
    results = model.fit(disp=0)  # Disable output during fitting
    
    # Predict probabilities
    df['MLR_fit'] = results.predict(X)
    df.to_csv('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/data/Well_B_MLR_fit.csv', index=False, encoding='utf-8-sig')
    
    # Create formula
    intercept = results.params[0]
    slope = results.params[1]
    formula = f"MLR_fit = {intercept:.1f} + {slope:.1f} * {mrl_column}"

    # Plot the fit
    plt.figure(figsize=(10, 6))
    plt.scatter(df[mrl_column], df[target_column], color='blue', label='LostCirculation', alpha=0.6)
    plt.scatter(df[mrl_column], df['MLR_fit'], color='red', label='MLR_fit', alpha=0.6)
    
    # Add title and labels
    plt.title('Logistic Fit')
    plt.xlabel('MLR_fit')
    plt.ylabel('Probability of Lost Circulation')
    plt.legend()
    plt.grid()
    
    # Display the logistic regression formula
    plt.text(0.05, 0.9, formula, transform=plt.gca().transAxes,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    plt.tight_layout()
    plt.savefig('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/mlr_fit.png', dpi=300)
    plt.show()

    # Calculate confusion matrix
    df['predicted'] = (df['MLR_fit'] >= threshold).astype(int)
    cm = confusion_matrix(df[target_column], df['predicted'])
    print("Confusion Matrix:")
    print(cm)

    # Calculate AUC
    auc = roc_auc_score(df[target_column], df['MLR_fit'])
    print(f"AUC: {auc:.3f}")

    # Plot ROC curve
    fpr, tpr, _ = roc_curve(df[target_column], df['MLR_fit'])
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC of MLR_fit = {auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')  # 45-degree line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve of MLR_fit')
    plt.legend(loc='lower right')
    plt.grid()
    plt.tight_layout()
    plt.savefig('E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/mlr_fit_roc.png', dpi=300)
    plt.show()
    return results  # Return the fit result

# Use the function
fit_logistic_model_and_plot(df, 'MLR', 'LostCirculation')

# Define the MLR_fit column
original_mlr_fit_zero = df[df['LostCirculation'] == 0]['MLR_fit']
original_mlr_fit_one = df[df['LostCirculation'] == 1]['MLR_fit']

# Call the function to draw the MLR_fit-LC comparison
threshold_value = 0.2  # Set threshold
save_path = 'E:/jupyter/lost_circulation/records/paper-bhyt/MLR/picture/comparison_mlr_fit.png'
plot_original_mlr_distribution(original_mlr_fit_zero, original_mlr_fit_one, threshold_value, save_path)
